In [1]:
import pandas as pd
import timeit
import numpy as np

#Reading Dataset
df = pd.read_csv('adult.data', sep=",", header = None , na_values = "?")
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [2]:
#Adding column names in the Dataset
df.columns = ['age', 'workclass','fnlwgt','education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race',
              'sex', 'capital_gain', 'capital_loss','hours_per_week','native_country', 'class']

In [3]:
#Preprocessing
#Converting continuous data into Categorical data
df['Age'] = pd.cut(x=df['age'], bins=[0, 18, 30, 50, 1000], labels=['Underage', 'Young', 'Adult','Elderly'])
#In this I have divided Age into 4 categories
#Similarly done for hours per week
df['Hours_per_Week'] = pd.cut(x=df['hours_per_week'], bins=[0, 20, 40, 1000], labels=['Part-Time', 'Full-Time', 'Overtime'])
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Age,Hours_per_Week
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,Adult,Full-Time
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,Adult,Part-Time
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,Adult,Full-Time
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,Elderly,Full-Time
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,Young,Full-Time


In [4]:
df = df.applymap(lambda space: space.strip() if type(space) is str else space)#Some columns like fnlwgt, education_num didn't give any perspective.
#Removed old age and hours_per_week columns
#Capital_Gain and Capital_Loss had many 0 values.
df = df.drop(['age','fnlwgt','education_num','hours_per_week','capital_loss', 'capital_gain'], axis = 1)
df.head() 

,workclass,education,marital_status,occupation,relationship,race,sex,native_country,class,Age,Hours_per_Week
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K,Adult,Full-Time
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K,Adult,Part-Time
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K,Adult,Full-Time
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K,Elderly,Full-Time
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K,Young,Full-Time


In [5]:
#Removing space from the each item.
df = df.applymap(lambda space: space.strip() if type(space) is str else space)
#Here Preprocessing and Cleaning is completed

Improved Apriori using Sampling

In [6]:
#Improvement using Sampling
#Taking a random sample containing 60% of original data
df = df.sample(frac=0.6)
df.describe()

,workclass,education,marital_status,occupation,relationship,race,sex,native_country,class,Age,Hours_per_Week
count,19537,19537,19537,19537,19537,19537,19537,19537,19537,19537,19537
unique,9,16,7,15,6,5,2,42,2,4,3
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K,Adult,Full-Time
freq,13620,6353,9035,2512,7959,16645,13146,17447,14850,9332,12051


In [31]:
#We will start the timer for Apriori Algorithm here
start_time = timeit.default_timer()
# Earlier we cleaned the dataframe. Now we will be using dataframe.values.tolist() to load the dataset.
def load_df():
    return df.values.tolist()
#First step is generating Candidate1.
def gen_cand1(itemset):
    CANDIDATE1 = []
    for i in itemset:
        for j in i:
            if not [j] in CANDIDATE1:
                CANDIDATE1.append([j])
    return list(map(frozenset, CANDIDATE1)) 
#Set in unhashable therefore we use frozenset which is nothing but immutable version of python set object.

itemset = load_df()
CANDIDATE1 = gen_cand1(itemset)

In [32]:
# Scanning Databse
def database_scan(Db, Ck, min_sup):
    sup_count = {}
    sup_data = {}
    r_list = []
    Db_length = len(Db)
    for t in Db:
        for i in Ck:
            if i.issubset(t):
                if not i in sup_count: sup_count[i]=1
                else: sup_count[i] += 1
    
    total_items = int(Db_length)
    for key in sup_count:
        support = sup_count[key]/total_items
        if support >= min_sup:
            r_list.insert(0,key)
        sup_data[key] = support
    return r_list, sup_data

#In this step we use our knowledge of support count. 
#If support of an item is greater than min_support, we insert that item and store it.

# Generating Apriori
def generate_apriori(L_k, k): 
    Ck = []
    for l in range(len(L_k)):
        for i in range(l+1, len(L_k)): 
            l1 = list(L_k[l])[:k-2]
            l2 = list(L_k[i])[:k-2]
            l1.sort()
            l2.sort()
            if l1==l2: 
                Ck.append(L_k[l] | L_k[i]) 
    return Ck

In [33]:
# Apriori function in which we specify min_sup and obtain frequent itemsets corresponding to it.
def apriori(itemset, min_sup = 0.12):
    CANDIDATE1 = gen_cand1(itemset)
    D = list(map(set, itemset))
    L1, sup_data = database_scan(D, CANDIDATE1, min_sup)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = generate_apriori(L[k-2], k)
        L_k, sup = database_scan(D, Ck, min_sup)
        sup_data.update(sup)
        L.append(L_k)
        k += 1
    return L, sup_data
L,sdata = apriori(itemset)
new_list = []
for index in range(len(L)):
    print("\nL{} ".format(index))
    print("Number of patterns={} \n".format(len(L[index])))
    improve_apriori_freq_pattern = [list(i) for i in L[index]]
    print(improve_apriori_freq_pattern)


L0 
Number of patterns=24 

[['Prof-specialty'], ['Some-college'], ['>50K'], ['Exec-managerial'], ['Divorced'], ['Craft-repair'], ['Bachelors'], ['Female'], ['Young'], ['Own-child'], ['HS-grad'], ['Overtime'], ['Adult'], ['Husband'], ['Married-civ-spouse'], ['Private'], ['Full-Time'], ['Elderly'], ['<=50K'], ['United-States'], ['Male'], ['White'], ['Not-in-family'], ['Never-married']]

L1 
Number of patterns=106 

[['Full-Time', 'Some-college'], ['White', 'Some-college'], ['Male', 'Some-college'], ['United-States', 'Some-college'], ['<=50K', 'Some-college'], ['Private', 'Some-college'], ['>50K', 'Male'], ['>50K', 'Husband'], ['>50K', 'White'], ['>50K', 'United-States'], ['>50K', 'Private'], ['>50K', 'Married-civ-spouse'], ['>50K', 'Adult'], ['Not-in-family', 'Private'], ['Female', 'Private'], ['Full-Time', 'Husband'], ['Full-Time', 'Adult'], ['Female', 'Never-married'], ['Female', 'Adult'], ['Divorced', 'United-States'], ['<=50K', 'Divorced'], ['Husband', 'United-States'], ['Adult', '

In [34]:
end_time = timeit.default_timer()
total_time = end_time - start_time
total_time

8.635148800000024